In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense

# Step 1: Data Preprocessing
# Load the Titanic dataset
titanic_data = pd.read_csv("titanic.csv")

# Remove unnecessary columns
unnecessary_columns = ['PassengerId', 'Name', 'Ticket', 'Cabin']
titanic_data.drop(columns=unnecessary_columns, inplace=True)

# Handle missing values
numeric_features = ['Age', 'Fare']
categorical_features = ['Sex', 'Embarked']
numeric_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Convert categorical data into numeric format
titanic_data_processed = preprocessor.fit_transform(titanic_data)
# Convert the processed data back into a DataFrame
titanic_df = pd.DataFrame(titanic_data_processed)

# Step 2: Normalization
# Since we are using Keras, normalization might be necessary. Let's normalize the numeric features.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
titanic_df[numeric_features] = scaler.fit_transform(titanic_df[numeric_features])

# Step 3: Model Training
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(titanic_df, titanic_data['Survived'], test_size=0.2, random_state=42)

# Define a function to create the Keras model
def create_model(input_dim, hidden_layers, units):
    model = Sequential()
    model.add(Dense(units, input_dim=input_dim, activation='relu'))
    for _ in range(hidden_layers):
        model.add(Dense(units, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define different architectures for the neural network
architectures = [(1, 64), (2, 32), (3, 16)]  # (number of hidden layers, units)

models = []
for arch in architectures:
    model = create_model(input_dim=X_train.shape[1], hidden_layers=arch[0], units=arch[1])
    models.append(model)

# Train the models
for model in models:
    model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=0)

# Step 4: Accuracy Calculation
# Calculate the accuracy of all models on the testing set
keras_accuracies = []
for model in models:
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    keras_accuracies.append(accuracy)

# Plot a bar graph to visually compare the accuracies of different neural network architectures
import matplotlib.pyplot as plt

plt.bar(range(len(architectures)), keras_accuracies, align='center')
plt.xticks(range(len(architectures)), architectures)
plt.xlabel('Neural Network Architectures')
plt.ylabel('Accuracy')
plt.title('Accuracy of Different Neural Network Architectures')
plt.show()
